# DataEng: Data Integration Activity

Name: Karan Patel

In [1]:
import pandas as pd
import numpy as np

## Aggregate Census Data to County Level

Create a python program that produces a __one-row-per-county__ version of the ACS data set. To do this you will need to think about how to properly aggregate Census Tract-level data into County-level summaries. 

In this step you can also eliminate unneeded columns from the ACS data. 

In [2]:
original_acs_df = pd.read_csv('./data/acs2017_census_tract_data.csv.gz', usecols=['State', 'County', 'TotalPop', 'IncomePerCap', 'Poverty'])
display(original_acs_df.head())

,State,County,TotalPop,IncomePerCap,Poverty
0,Alabama,Autauga County,1845,33018.0,10.7
1,Alabama,Autauga County,2172,18996.0,22.4
2,Alabama,Autauga County,3385,21236.0,14.7
3,Alabama,Autauga County,4267,28068.0,2.3
4,Alabama,Autauga County,9965,36905.0,12.2


__Question__: Show your aggregated county-level data rows for the following counties: Loudoun County Virginia, Washington County Oregon, Harlan County Kentucky, Malheur County oregon

__Answer__: See output from code block below.

In [3]:
def acs_data_county_aggregator(df):
    total_pop = df['TotalPop'].sum()
    
    data = {'TotalPop': total_pop.astype(int),
            'IncomePerCap': ((df['IncomePerCap'] * df['TotalPop']).sum() / total_pop).round(2),
            'Poverty': ((df['Poverty'] * df['TotalPop']).sum() / total_pop).round(2)
            }
    return pd.Series(data)

acs_df = original_acs_df.groupby(['State', 'County']).apply(acs_data_county_aggregator).reset_index()

def get_criteria(county, state, df):
    return (df.County == county) & (df.State == state)

display(acs_df[get_criteria('Loudoun County', 'Virginia', acs_df)])
display(acs_df[get_criteria('Washington County', 'Oregon', acs_df)])
display(acs_df[get_criteria('Harlan County', 'Kentucky', acs_df)])
display(acs_df[get_criteria('Malheur County', 'Oregon', acs_df)])

,State,County,TotalPop,IncomePerCap,Poverty
2968,Virginia,Loudoun County,374558.0,50455.65,3.69


,State,County,TotalPop,IncomePerCap,Poverty
2241,Oregon,Washington County,572071.0,35369.05,10.32


,State,County,TotalPop,IncomePerCap,Poverty
1040,Kentucky,Harlan County,27548.0,15456.97,35.67


,State,County,TotalPop,IncomePerCap,Poverty
2230,Oregon,Malheur County,30421.0,17567.5,24.3


## Simplify the COVID Data

Simplify the COVID data along the time dimension. The COVID data set contains day-level resolution data from (approximately) March of 2020 through February of 2021. However, you will only need four data points per county: total cases, total deaths, cases reported during December of 2020 and deaths reported during December 2020. 

Create a python program that reduces the COVID data to one line per county. 


In [4]:
original_covid_df = pd.read_csv('./data/COVID_county_data.csv.gz', parse_dates=['date'])
display(original_covid_df.head())

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


__Question__: Show your simplified COVID data for the counties listed above. 

__Answer:__

In [18]:
def covid_data_county_aggregator(df):
    data = {'total-cases': df['cases'].sum().astype(int),
            'total-deaths': df['deaths'].sum().astype(int),
            'cases-dec-2020': df[df['date'].between('12-1-2020', '12-31-2020')]['cases'].sum().astype(int),
            'deaths-dec-2020': df[df['date'].between('12-1-2020', '12-31-2020')]['deaths'].sum().astype(int)
            }
    return pd.Series(data)

covid_df = original_covid_df.groupby(['state', 'county']).apply(covid_data_county_aggregator).reset_index()

def get_criteria(county, state, df):
    return (df.county == county) & (df.state == state)

display(covid_df[get_criteria('Loudoun', 'Virginia', covid_df)])
display(covid_df[get_criteria('Washington', 'Oregon', covid_df)])
display(covid_df[get_criteria('Harlan', 'Kentucky', covid_df)])
display(covid_df[get_criteria('Malheur', 'Oregon', covid_df)])

,state,county,total-cases,total-deaths,cases-dec-2020,deaths-dec-2020
3017,Virginia,Loudoun,2496450,35820,376223,4729


,state,county,total-cases,total-deaths,cases-dec-2020,deaths-dec-2020
2277,Oregon,Washington,2157339,22455,424620,3860


,state,county,total-cases,total-deaths,cases-dec-2020,deaths-dec-2020
1054,Kentucky,Harlan,205984,3994,38959,506


,state,county,total-cases,total-deaths,cases-dec-2020,deaths-dec-2020
2265,Oregon,Malheur,453634,7770,82916,1465


## Integrate COVID Data with ACS Data

Create a single pandas DataFrame containing one row per county and using the columns described above. You are free to add additional columns if needed. For example, you might want to normalize all of the COVID data by the population of each county so that you have a consistent “number of cases/deaths per 100000 residents” value for each county.

__Question:__ List your integrated data for all counties in the State of Oregon.

__Answer__:

In [6]:
acs_df_or = acs_df[acs_df.State == 'Oregon']
acs_df_or['County'] = acs_df_or['County'].str.replace(' County', '')

covid_df_or = covid_df[covid_df.state == 'Oregon']

or_df = pd.merge(acs_df_or, covid_df_or, left_on = ['State', 'County'], right_on = ['state', 'county'])

or_df = or_df.drop(columns=['state'])
or_df = or_df.drop(columns=['county'])

display(or_df)

<ipython-input-6-1ec1bf73f918>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs_df_or['County'] = acs_df_or['County'].str.replace(' County', '')


,State,County,TotalPop,IncomePerCap,Poverty,total-cases,total-deaths,cases-dec-2020,deaths-dec-2020
0,Oregon,Baker,15980.0,25820.27,15.08,55586,663,11688,133
1,Oregon,Benton,88249.0,30872.82,22.42,180225,2304,34260,278
2,Oregon,Clackamas,399962.0,37550.85,8.98,1284402,20040,261810,3125
3,Oregon,Clatsop,38021.0,28114.63,12.19,77666,287,14439,47
4,Oregon,Columbia,50207.0,28459.69,12.32,105324,1363,21459,266
5,Oregon,Coos,62921.0,26007.21,17.90,100097,969,18806,151
6,Oregon,Crook,21717.0,24238.81,15.32,55863,1134,11048,196
7,Oregon,Curry,22377.0,26925.54,15.41,30045,393,6741,72
8,Oregon,Deschutes,175321.0,31574.93,12.10,509974,4141,102490,563
9,Oregon,Douglas,107576.0,25001.73,17.03,174952,3983,37590,964


## Analysis

For each of the following, determine the strength of the correlation between each pair of variables. Compute the correlation strength by calculating the Pearson correlation coefficient R for pairs of columns in your DataFrame. For example, if you have a DataFrame df with each row representing a distinct county, and columns named ‘TotalCases’ and ‘Poverty’, then you can compute R like this:

> R = df[‘TotalCases’].corr(df[‘Poverty’])

For any R that is > 0.5 or < -0.5 also display a scatter plot (see pandas scatterplot and seaborn documentation for information about how to display scatter plots from DataFrame data).

The COVID numbers should be normalized to population (# of cases per 100,000 residents) so that different sized counties are comparable. So for example, “COVID total cases” below really means “((COVID total cases in county * 100000) / population of county)”.

Across all of the counties in the State of Oregon:
* COVID total cases vs. % population in poverty

In [7]:
total_cases = (or_df['total-cases'] * 100000) / or_df['TotalPop']

r = total_cases.corr(or_df['Poverty'])
print(f'r = {r}')

r = 0.2870979669130601


* COVID total deaths vs. % population in poverty

In [8]:
total_deaths = (or_df['total-deaths'] * 100000) / or_df['TotalPop']

r = total_deaths.corr(or_df['Poverty'])
print(f'r = {r}')

r = 0.3605381417925185


* COVID total cases vs. Per Capita Income level

In [9]:
total_cases = (or_df['total-cases'] * 100000) / or_df['TotalPop']

r = total_cases.corr(or_df['IncomePerCap'])
print(f'r = {r}')

r = -0.37568496125710993


* COVID total deaths vs. Per Capita Income level

In [10]:
total_deaths = (or_df['total-deaths'] * 100000) / or_df['TotalPop']

r = total_deaths.corr(or_df['IncomePerCap'])
print(f'r = {r}')

r = -0.46186658814724785


* COVID cases during December 2020 vs. % population in poverty

In [11]:
total_cases = (or_df['cases-dec-2020'] * 100000) / or_df['TotalPop']

r = total_cases.corr(or_df['Poverty'])
print(f'r = {r}')

r = 0.2981575324663765


* COVID deaths during December 2020 vs. % population in poverty

In [12]:
total_deaths = (or_df['deaths-dec-2020'] * 100000) / or_df['TotalPop']

r = total_deaths.corr(or_df['Poverty'])
print(f'r = {r}')

r = 0.30275469931451643


* COVID cases during December 2020 vs. Per Capita Income level

In [13]:
total_cases = (or_df['cases-dec-2020'] * 100000) / or_df['TotalPop']

r = total_cases.corr(or_df['IncomePerCap'])
print(f'r = {r}')

r = -0.3853971317908648


__Across all of the counties in the entire USA__

In [14]:
# create df for all us counties 
acs_df['County'] = acs_df['County'].str.replace(' County', '')

us_df = pd.merge(acs_df, covid_df, left_on = ['State', 'County'], right_on = ['state', 'county'])
us_df = us_df.drop(columns=['state'])
us_df = us_df.drop(columns=['county'])
display(us_df)

,State,County,TotalPop,IncomePerCap,Poverty,total-cases,total-deaths,cases-dec-2020,deaths-dec-2020
0,Alabama,Autauga,55036.0,27823.92,13.76,645935,9042,108652,1355
1,Alabama,Baldwin,203360.0,29364.37,11.87,2003567,23041,348455,4502
2,Alabama,Barbour,26201.0,17561.09,26.87,268771,4077,40753,931
3,Alabama,Bibb,22580.0,20911.18,14.92,261043,5272,47009,1244
4,Alabama,Blount,57667.0,22020.72,15.60,630106,8669,121270,1590
...,...,...,...,...,...,...,...,...,...
3063,Wyoming,Sweetwater,44527.0,31699.95,12.08,323730,2239,78122,412
3064,Wyoming,Teton,22923.0,49200.63,6.84,305376,617,59845,67
3065,Wyoming,Uinta,20758.0,27114.84,14.82,200783,1037,41859,201
3066,Wyoming,Washakie,8253.0,27344.99,12.87,84354,2622,20107,350


In [19]:
# COVID total cases vs. % population in poverty
total_cases = (us_df['total-cases'] * 100000) / us_df['TotalPop']
r = total_cases.corr(us_df['Poverty'])
print(f'r = {r}')

# COVID total deaths vs. % population in poverty
total_deaths = (us_df['total-deaths'] * 100000) / us_df['TotalPop']
r = total_deaths.corr(us_df['Poverty'])
print(f'r = {r}')

# COVID total cases vs. Per Capita Income level
total_cases = (us_df['total-cases'] * 100000) / us_df['TotalPop']
r = total_cases.corr(us_df['IncomePerCap'])
print(f'r = {r}')

# COVID total deaths vs. Per Capita Income level
total_deaths = (us_df['total-deaths'] * 100000) / us_df['TotalPop']
r = total_deaths.corr(us_df['IncomePerCap'])
print(f'r = {r}')

# COVID cases during December 2020 vs. % population in poverty
total_cases_dec_2020 = (us_df['cases-dec-2020'] * 100000) / us_df['TotalPop']
r = total_cases_dec_2020.corr(us_df['Poverty'])
print(f'r = {r}')

# COVID deaths during December 2020 vs. % population in poverty
total_deaths_dec_2020 = (us_df['cases-dec-2020'] * 100000) / us_df['TotalPop']
r = total_deaths_dec_2020.corr(us_df['Poverty'])
print(f'r = {r}')

# COVID cases during December 2020 vs. Per Capita Income level
total_cases_dec_2020 = (us_df['cases-dec-2020'] * 100000) / us_df['TotalPop']
r = total_deaths_dec_2020.corr(us_df['IncomePerCap'])
print(f'r = {r}')

r = 0.19742929493170183
r = 0.26803430608102446
r = -0.21366063002097785
r = -0.17345835459087575
r = 0.07035278331532802
r = 0.07035278331532802
r = -0.1593355765234985
